In [2]:
import os
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from batchgenerators.utilities.file_and_folder_operations import join
from nnunetv2.inference.predict_from_raw_data import nnUNetPredictor
from PIL import Image, ImageFile
from psd_tools import PSDImage
import csv
import re

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="psd_tools") # PSDToolsの警告を無視
warnings.filterwarnings("ignore", message="No label manager specified in plans") # nnUNetの警告を無視

In [45]:
# 環境変数を設定
os.environ['nnUNet_raw'] = '/Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data'
os.environ['nnUNet_results'] = '/Users/yuma/Yuma-Kanematsu/nnUNet/src/results'
os.environ['nnUNet_preprocessed'] = '/Users/yuma/Yuma-Kanematsu/nnUNet/src/preprocessed_data' 

# 画像に関する問題を回避するための設定
ImageFile.LOAD_TRUNCATED_IMAGES = True

# パスの設定
model_folder = '/Users/yuma/Yuma-Kanematsu/nnUNet/src/results/Dataset004_Orbital_same_testdata/nnUNetTrainer__nnUNetPlans__2d'
input_folder = '/Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset004_Orbital_same_testdata/imagesTs'

output_folder = '/Users/yuma/Yuma-Kanematsu/nnUNet/src/results/250419_inference'
csv_output_path = os.path.join(output_folder, 'evaluation_metrics.csv') # 評価結果を保存するCSVファイルのパス
visualization_folder = os.path.join(output_folder, 'visualizations') # 可視化画像の保存フォルダ

# 可視化フォルダが存在しない場合は作成
os.makedirs(visualization_folder, exist_ok=True)

# メトリクス計算関数
def calculate_metrics(pred_mask, gt_mask):
    """
    予測マスクと正解マスクからDice、IoU、Accuracy、Precision、Recallを計算する関数
    
    Parameters:
    pred_mask (numpy.ndarray): 予測マスク
    gt_mask (numpy.ndarray): 正解マスク
    
    Returns:
    tuple: (dice, iou, accuracy, precision, recall)
    """
    # マスクの値を確認し、デバッグ情報を出力
    print(f"予測マスクの値範囲: {np.min(pred_mask)} 〜 {np.max(pred_mask)}")
    print(f"正解マスクの値範囲: {np.min(gt_mask)} 〜 {np.max(gt_mask)}")
    
    # 2値画像に変換
    pred_binary = (pred_mask > 0).astype(np.uint8)
    gt_binary = (gt_mask > 0).astype(np.uint8)
    
    # 交差（Intersection）
    intersection = np.logical_and(pred_binary, gt_binary).sum()
    
    # 合併（Union）
    union = np.logical_or(pred_binary, gt_binary).sum()
    
    # デバッグ情報を出力
    print(f"Intersection: {intersection}, Union: {union}")
    print(f"予測マスクの陽性ピクセル数: {pred_binary.sum()}")
    print(f"正解マスクの陽性ピクセル数: {gt_binary.sum()}")
    
    # Precision（適合率）: TP / (TP + FP)
    precision = intersection / (pred_binary.sum() + 1e-10)
    
    # Recall（再現率）: TP / (TP + FN)
    recall = intersection / (gt_binary.sum() + 1e-10)
    
    # Dice係数
    dice = (2.0 * intersection) / (pred_binary.sum() + gt_binary.sum() + 1e-10) # ゼロ除算エラーを防ぐための小さな値（1e-10）を追加
    
    # IoU（Intersection over Union）
    iou = intersection / (union + 1e-10)
    
    # Accuracy
    total_pixels = pred_binary.size
    correct_pixels = (pred_binary == gt_binary).sum()
    accuracy = correct_pixels / total_pixels
    
    return dice, iou, accuracy, precision, recall, pred_binary, gt_binary

# PSDファイルからマスク画像を抽出する関数
def extract_mask_from_psd(psd_path, color_threshold=50):
    """
    PSDファイルからマスク画像を赤チャンネルの閾値に基づいて抽出する関数
    
    Parameters:
    psd_path (str): PSDファイルのパス
    color_threshold (int): 赤チャンネルの閾値
    
    Returns:
    numpy.ndarray: 抽出したマスク画像
    """
    try:
        print(f"PSDファイルを読み込みました: {psd_path}")
        
        # Open PSD file
        psd = PSDImage.open(psd_path)
        
        # Get composite image
        composite_image = psd.composite()
        
        # Convert to numpy array
        img_array = np.array(composite_image)
        print(f"PSD画像配列形状: {img_array.shape}, 型: {img_array.dtype}")
        
        # Extract red channel
        if len(img_array.shape) == 3 and img_array.shape[2] == 4:  # RGBA
            print("RGBAフォーマットを処理中")
            red_channel = img_array[:, :, 0]
            green_channel = img_array[:, :, 1]
            blue_channel = img_array[:, :, 2]
            alpha_channel = img_array[:, :, 3]
            
            # Red mask condition: high red component, low green and blue, has transparency
            red_mask = ((red_channel > green_channel + color_threshold) & 
                        (red_channel > blue_channel + color_threshold) & 
                        (alpha_channel > 0))
        elif len(img_array.shape) == 3 and img_array.shape[2] == 3:  # RGB
            print("RGBフォーマットを処理中")
            red_channel = img_array[:, :, 0]
            green_channel = img_array[:, :, 1]
            blue_channel = img_array[:, :, 2]
            
            # Red mask condition
            red_mask = ((red_channel > green_channel + color_threshold) & 
                        (red_channel > blue_channel + color_threshold))
        else:
            # Grayscale - すべて陽性として扱うか、別のロジックが必要
            print(f"グレースケールまたは非対応フォーマット (形状: {img_array.shape})")
            if len(img_array.shape) == 2:
                # グレースケール画像の場合は単純に0より大きい値を陽性とする
                red_mask = img_array > 0
            else:
                print("非対応の画像形式です")
                return None
        
        # Convert to binary mask (0 or 1)
        binary_mask = red_mask.astype(np.uint8)
        
        return binary_mask
    except Exception as e:
        print(f"PSDファイル処理エラー: {psd_path}, エラー: {e}")
        import traceback
        traceback.print_exc()
        return None

# 可視化関数
def create_visualization(original_image_path, pred_mask, gt_mask, output_path, title=""):
    """
    元画像、予測マスク、正解マスクの可視化画像を作成して保存する関数
    
    Parameters:
    original_image_path (str): 元画像のパス
    pred_mask (numpy.ndarray): 予測マスク（バイナリ）
    gt_mask (numpy.ndarray): 正解マスク（バイナリ）
    output_path (str): 保存先のパス
    title (str): 画像タイトル
    """
    try:
        # 元画像の読み込み
        original_img = Image.open(original_image_path)
        original_array = np.array(original_img)
        
        # 予測マスクと正解マスクを255にスケーリング（表示用）
        pred_display = pred_mask * 255
        gt_display = gt_mask * 255
        
        # 元画像がグレースケールの場合はRGBに変換
        if len(original_array.shape) == 2:
            original_rgb = np.stack([original_array, original_array, original_array], axis=2)
        else:
            original_rgb = original_array.copy()
        
        # オーバーレイ画像の作成（元画像 + 予測マスク）
        overlay_img = original_rgb.copy()
        
        # 赤色で予測マスクを表示
        if len(overlay_img.shape) == 3 and overlay_img.shape[2] >= 3:
            overlay_img[pred_mask > 0, 0] = 255  # Red channel
            overlay_img[pred_mask > 0, 1] = 0    # Green channel
            overlay_img[pred_mask > 0, 2] = 0    # Blue channel
        
        # 2x2のグリッドで可視化
        fig, axes = plt.subplots(2, 2, figsize=(12, 10))
        
        # 正解マスク
        axes[0, 0].imshow(gt_display, cmap='gray')
        axes[0, 0].set_title('Ground Truth')
        axes[0, 0].axis('off')
        
        # 予測マスク
        axes[0, 1].imshow(pred_display, cmap='gray')
        axes[0, 1].set_title('Predicted Mask')
        axes[0, 1].axis('off')
        
        # 元画像
        axes[1, 0].imshow(original_array, cmap='gray' if len(original_array.shape) == 2 else None)
        axes[1, 0].set_title('Original Image')
        axes[1, 0].axis('off')
        
        # オーバーレイ（元画像 + 予測マスク）
        axes[1, 1].imshow(overlay_img)
        axes[1, 1].set_title('Original with Predicted Mask')
        axes[1, 1].axis('off')
        
        # タイトルがあれば設定
        if title:
            fig.suptitle(title, fontsize=16)
            
        # レイアウト調整と保存
        plt.tight_layout()
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        plt.close(fig)  # メモリ解放のためにfigureを閉じる
        
        print(f"可視化画像を保存しました: {output_path}")
        
    except Exception as e:
        print(f"可視化処理エラー: {e}")
        import traceback
        traceback.print_exc()

# ファイル名から患者IDと画像番号を抽出する関数
def extract_patient_info(filename):
    """
    ファイル名から患者IDと画像番号を抽出する関数
    
    Parameters:
    filename (str): ファイル名
    
    Returns:
    tuple: (patient_id, img_num)
    """
    # 正規表現でパターンを抽出
    pattern = r'(\d+)_(\d+)_'
    match = re.search(pattern, filename)
    if match:
        patient_id = match.group(1)
        img_num = match.group(2)
        return patient_id, img_num
    else:
        return None, None

# 1. 推論の実行
def run_inference():
    """
    nnUNetを使用して推論を実行する関数
    """
    predictor = nnUNetPredictor(
        tile_step_size=0.5,
        use_gaussian=True,
        use_mirroring=True,
        perform_everything_on_device=False,
        device=torch.device('mps', 0),
        verbose=False,
        verbose_preprocessing=False,
        allow_tqdm=True
    )
    
    # モデルの初期化
    predictor.initialize_from_trained_model_folder(
        model_folder,
        use_folds=(0,),
        checkpoint_name='checkpoint_best.pth',
    )
    
    # 推論を実行
    predictor.predict_from_files(
        input_folder,
        output_folder,
        save_probabilities=False, 
        overwrite=True,
        num_processes_preprocessing=1, 
        num_processes_segmentation_export=1,
        folder_with_segs_from_prev_stage=None, 
        num_parts=1, 
        part_id=0
    )
    print("推論が完了しました。")

# 2. 評価と可視化の実行
def evaluate_and_visualize_results():
    """
    推論結果と正解ラベルを比較して評価指標を計算し、可視化する関数
    """
    # 出力フォルダにあるファイルのリストを取得
    prediction_files = [f for f in os.listdir(output_folder) if f.endswith('.png')]
    
    # 評価結果を保存するリスト
    results = []
    
    for pred_file in prediction_files:
        patient_id, img_num = extract_patient_info(pred_file)
        
        if patient_id is None:
            print(f"ファイル名からの情報抽出に失敗: {pred_file}")
            continue
        
        # 対応するPSDファイルを探す
        # もしマッチするファイルが見つからない場合、より一般的な方法で検索

        # 患者IDと画像番号で始まるPSDファイルを探す
        prefix = f"{patient_id}_{img_num}_"
        matching_files = [f for f in os.listdir(input_folder) if f.startswith(prefix) and f.endswith('.psd')]
        
        if matching_files:
            psd_path = os.path.join(input_folder, matching_files[0])
            print(f"代替のPSDファイルが見つかりました: {matching_files[0]}")
        else:
            print(f"対応するPSDファイルが見つかりません: {patient_id}_{img_num}_*.psd")
            continue
        
        # 元画像のパスを作成（通常は入力フォルダの中の[patient_id]_[img_num]_*.pngという形式）
        original_image_pattern = f"{patient_id}_{img_num}_R_FLOOR-*_0000.png"
        matching_originals = [f for f in os.listdir(input_folder) if re.match(f"{patient_id}_{img_num}_.*_0000\.png", f)]
        
        if not matching_originals:
            print(f"元画像が見つかりません: {original_image_pattern}")
            continue
            
        original_image_path = os.path.join(input_folder, matching_originals[0])
        
        # 予測マスクの読み込み
        pred_path = os.path.join(output_folder, pred_file)
        pred_mask = np.array(Image.open(pred_path))
        
        # 正解マスクの読み込み
        gt_mask = extract_mask_from_psd(psd_path)
        
        if gt_mask is None:
            print(f"正解マスクの抽出に失敗: {psd_path}")
            continue
        
        # サイズが異なる場合はリサイズ
        if pred_mask.shape != gt_mask.shape:
            print(f"マスクサイズが異なります。予測: {pred_mask.shape}, 正解: {gt_mask.shape}")
            
            # リサイズ処理
            gt_mask_pil = Image.fromarray(gt_mask)
            gt_mask_pil = gt_mask_pil.resize((pred_mask.shape[1], pred_mask.shape[0]), Image.NEAREST)
            gt_mask = np.array(gt_mask_pil)
        
        # メトリクスの計算
        dice, iou, accuracy, precision, recall, pred_binary, gt_binary = calculate_metrics(pred_mask, gt_mask)
        
        # 結果を追加
        results.append({
            'Patient_ID': patient_id,
            'img_num': img_num,
            'IoU': iou,
            'Dice': dice,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
        })
        
        print(f"処理完了: {pred_file}, Dice: {dice:.4f}, IoU: {iou:.4f}, Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")
        
        # 可視化画像の保存パス
        vis_filename = f"{patient_id}_{img_num}_visualization.png"
        vis_path = os.path.join(visualization_folder, vis_filename)
        
        # 可視化を実行
        title = f"Patient: {patient_id}, Image: {img_num}, Dice: {dice:.4f}, IoU: {iou:.4f}"
        create_visualization(original_image_path, pred_binary, gt_binary, vis_path, title)
    
    # 結果をCSVに出力
    df = pd.DataFrame(results)
    df.to_csv(csv_output_path, index=False)
    print(f"評価結果をCSVに保存しました: {csv_output_path}")

# メイン処理
if __name__ == "__main__":
    # 1. 推論の実行
    print("推論を開始します...")
    run_inference()
    
    # 2. 評価と可視化の実行
    print("評価と可視化を開始します...")
    evaluate_and_visualize_results()
    
    print("すべての処理が完了しました。")

推論を開始します...
perform_everything_on_device=True is only supported for cuda devices! Setting this to False
There are 19 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 19 cases that I would like to predict

Predicting 6870292_16_R_FLOOR-2:
perform_everything_on_device: False


100%|██████████| 2/2 [00:00<00:00,  7.80it/s]


sending off prediction to background worker for resampling and export
done with 6870292_16_R_FLOOR-2

Predicting 6870292_19_R_FLOOR-3:
perform_everything_on_device: False


100%|██████████| 2/2 [00:00<00:00,  9.01it/s]


sending off prediction to background worker for resampling and export
done with 6870292_19_R_FLOOR-3

Predicting 6870292_20_R_FLOOR-3:
perform_everything_on_device: False


...

可視化画像を保存しました: /Users/yuma/Yuma-Kanematsu/nnUNet/src/results/250419_inference/visualizations/6887036_14_visualization.png
代替のPSDファイルが見つかりました: 6955010_23_R_FLOOR-3.psd
PSDファイルを読み込みました: /Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset004_Orbital_same_testdata/imagesTs/6955010_23_R_FLOOR-3.psd
PSD画像配列形状: (512, 512, 4), 型: uint8
RGBAフォーマットを処理中
予測マスクの値範囲: 0 〜 1
正解マスクの値範囲: 0 〜 1
Intersection: 1141, Union: 1756
予測マスクの陽性ピクセル数: 1450
正解マスクの陽性ピクセル数: 1447
処理完了: 6955010_23_R_FLOOR-3.png, Dice: 0.7877, IoU: 0.6498, Accuracy: 0.9977, Precision: 0.7869, Recall: 0.7885


Unknown image resource 1092
Unknown key: b'CAI '
Unknown tagged block: b'CAI ', b'\x00\x00\x00\x03\x00\x00\x00\x10\x00\x00\x00\x01\x00\x00\x00\x00 ... =77'
Unknown key: b'OCIO'
Unknown tagged block: b'OCIO', b'\x00\x00\x00\x10\x00\x00\x00\x01\x00\x00\x00\x00\x00\x1bdo ... =170'
Unknown key: b'GenI'
Unknown tagged block: b'GenI', b'\x00\x00\x00\x10\x00\x00\x00\x01\x00\x00\x00\x00\x00\x0bge ... =84'


可視化画像を保存しました: /Users/yuma/Yuma-Kanematsu/nnUNet/src/results/250419_inference/visualizations/6955010_23_visualization.png
代替のPSDファイルが見つかりました: 6870292_22_R_FLOOR-3.psd
PSDファイルを読み込みました: /Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset004_Orbital_same_testdata/imagesTs/6870292_22_R_FLOOR-3.psd
PSD画像配列形状: (512, 512, 4), 型: uint8
RGBAフォーマットを処理中
予測マスクの値範囲: 0 〜 1
正解マスクの値範囲: 0 〜 1
Intersection: 1818, Union: 2574
予測マスクの陽性ピクセル数: 2197
正解マスクの陽性ピクセル数: 2195
処理完了: 6870292_22_R_FLOOR-3.png, Dice: 0.8279, IoU: 0.7063, Accuracy: 0.9971, Precision: 0.8275, Recall: 0.8282


Unknown image resource 1092
Unknown key: b'CAI '
Unknown tagged block: b'CAI ', b'\x00\x00\x00\x03\x00\x00\x00\x10\x00\x00\x00\x01\x00\x00\x00\x00 ... =77'
Unknown key: b'OCIO'
Unknown tagged block: b'OCIO', b'\x00\x00\x00\x10\x00\x00\x00\x01\x00\x00\x00\x00\x00\x1bdo ... =170'
Unknown key: b'GenI'
Unknown tagged block: b'GenI', b'\x00\x00\x00\x10\x00\x00\x00\x01\x00\x00\x00\x00\x00\x0bge ... =84'


可視化画像を保存しました: /Users/yuma/Yuma-Kanematsu/nnUNet/src/results/250419_inference/visualizations/6870292_22_visualization.png
代替のPSDファイルが見つかりました: 6870292_21_R_FLOOR-3.psd
PSDファイルを読み込みました: /Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset004_Orbital_same_testdata/imagesTs/6870292_21_R_FLOOR-3.psd
PSD画像配列形状: (512, 512, 4), 型: uint8
RGBAフォーマットを処理中
予測マスクの値範囲: 0 〜 1
正解マスクの値範囲: 0 〜 1
Intersection: 2240, Union: 2909
予測マスクの陽性ピクセル数: 2617
正解マスクの陽性ピクセル数: 2532
処理完了: 6870292_21_R_FLOOR-3.png, Dice: 0.8701, IoU: 0.7700, Accuracy: 0.9974, Precision: 0.8559, Recall: 0.8847
可視化画像を保存しました: /Users/yuma/Yuma-Kanematsu/nnUNet/src/results/250419_inference/visualizations/6870292_21_visualization.png
評価結果をCSVに保存しました: /Users/yuma/Yuma-Kanematsu/nnUNet/src/results/250419_inference/evaluation_metrics.csv
すべての処理が完了しました。


In [4]:
import pandas as pd
from tabulate import tabulate

# CSVからデータを読み込む
print("CSVファイルを読み込みます...")
print(f"CSVファイルのパス: {csv_output_path}")
df_test = pd.read_csv(csv_output_path)

# 指標の平均値を計算
mean_dice = df_test['Dice'].mean() * 100  # パーセント表示に変換
mean_hd95 = df_test['HD95'].mean() if 'HD95' in df_test.columns else 0  # HD95カラムがある場合
mean_iou = df_test['IoU'].mean() * 100
mean_acc = df_test['Accuracy'].mean() * 100
mean_precision = df_test['Precision'].mean() * 100
mean_recall = df_test['Recall'].mean() * 100

# 表のデータを作成
data = [
    ['nnUNet', f"{mean_dice:.2f}%", f"{mean_iou:.2f}%", 
     f"{mean_acc:.2f}%", f"{mean_precision:.2f}%", f"{mean_recall:.2f}%"]
]

# モデルの種類を追加する場合は以下のようにdataに追加
# data.append(['別のモデル名', '値1', '値2', ...])

# 表のヘッダーを定義
headers = ["", "Dice", "IoU", "Accuracy", "Precision", "Recall"]

# 表を生成
table = tabulate(data, headers, tablefmt="grid")
print(table)

# 詳細な指標も出力
print(f"\n解析したテストデータの数: {len(df_test)}")
print(f"平均Precision: {mean_precision/100:.4f}")
print(f"平均Recall: {mean_recall/100:.4f}")
print(f"平均F1スコア: {2 * ((mean_precision/100) * (mean_recall/100)) / ((mean_precision/100) + (mean_recall/100)):.4f}")
print(f"平均Dice: {mean_dice/100:.4f}")
print(f"平均Accuracy: {mean_acc/100:.4f}")
print(f"平均IoU: {mean_iou/100:.4f}")

CSVファイルを読み込みます...
CSVファイルのパス: /Users/yuma/Yuma-Kanematsu/nnUNet/src/results/250422_inference_ROI/evaluation_metrics.csv
+--------+--------+--------+------------+-------------+----------+
|        | Dice   | IoU    | Accuracy   | Precision   | Recall   |
+========+========+========+============+=============+==========+
| nnUNet | 83.01% | 71.14% | 98.89%     | 83.52%      | 83.33%   |
+--------+--------+--------+------------+-------------+----------+

解析したテストデータの数: 19
平均Precision: 0.8352
平均Recall: 0.8333
平均F1スコア: 0.8343
平均Dice: 0.8301
平均Accuracy: 0.9889
平均IoU: 0.7114


In [3]:

# 環境変数を設定
os.environ['nnUNet_raw'] = '/Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data'
os.environ['nnUNet_results'] = '/Users/yuma/Yuma-Kanematsu/nnUNet/src/results'
os.environ['nnUNet_preprocessed'] = '/Users/yuma/Yuma-Kanematsu/nnUNet/src/preprocessed_data' 

# 画像に関する問題を回避するための設定
ImageFile.LOAD_TRUNCATED_IMAGES = True

# パスの設定
model_folder = '/Users/yuma/Yuma-Kanematsu/nnUNet/src/results/Dataset005_Orbital_ROI/nnUNetTrainer__nnUNetPlans__2d'
input_folder = '/Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset005_Orbital_ROI/imagesTs'
gt_mask_folder = '/Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset005_Orbital_ROI/labelsTs'  # 正解マスクの場所

output_folder = '/Users/yuma/Yuma-Kanematsu/nnUNet/src/results/250422_inference_ROI'
csv_output_path = os.path.join(output_folder, 'evaluation_metrics.csv') # 評価結果を保存するCSVファイルのパス
visualization_folder = os.path.join(output_folder, 'visualizations') # 可視化画像の保存フォルダ

# 可視化フォルダが存在しない場合は作成
os.makedirs(visualization_folder, exist_ok=True)

# メトリクス計算関数
def calculate_metrics(pred_mask, gt_mask):
    """
    予測マスクと正解マスクからDice、IoU、Accuracy、Precision、Recallを計算する関数
    
    Parameters:
    pred_mask (numpy.ndarray): 予測マスク
    gt_mask (numpy.ndarray): 正解マスク
    
    Returns:
    tuple: (dice, iou, accuracy, precision, recall, pred_binary, gt_binary)
    """
    # マスクの値を確認し、デバッグ情報を出力
    print(f"予測マスクの値範囲: {np.min(pred_mask)} 〜 {np.max(pred_mask)}")
    print(f"正解マスクの値範囲: {np.min(gt_mask)} 〜 {np.max(gt_mask)}")
    
    # 2値画像に変換
    pred_binary = (pred_mask > 0).astype(np.uint8)
    gt_binary = (gt_mask > 0).astype(np.uint8)
    
    # 交差（Intersection）
    intersection = np.logical_and(pred_binary, gt_binary).sum()
    
    # 合併（Union）
    union = np.logical_or(pred_binary, gt_binary).sum()
    
    # デバッグ情報を出力
    print(f"Intersection: {intersection}, Union: {union}")
    print(f"予測マスクの陽性ピクセル数: {pred_binary.sum()}")
    print(f"正解マスクの陽性ピクセル数: {gt_binary.sum()}")
    
    # Precision（適合率）: TP / (TP + FP)
    precision = intersection / (pred_binary.sum() + 1e-10)
    
    # Recall（再現率）: TP / (TP + FN)
    recall = intersection / (gt_binary.sum() + 1e-10)
    
    # Dice係数
    dice = (2.0 * intersection) / (pred_binary.sum() + gt_binary.sum() + 1e-10) # ゼロ除算エラーを防ぐための小さな値（1e-10）を追加
    
    # IoU（Intersection over Union）
    iou = intersection / (union + 1e-10)
    
    # Accuracy
    total_pixels = pred_binary.size
    correct_pixels = (pred_binary == gt_binary).sum()
    accuracy = correct_pixels / total_pixels
    
    return dice, iou, accuracy, precision, recall, pred_binary, gt_binary

# 可視化関数
def create_visualization(original_image_path, pred_mask, gt_mask, output_path, title=""):
    """
    元画像、予測マスク、正解マスクの可視化画像を作成して保存する関数
    
    Parameters:
    original_image_path (str): 元画像のパス
    pred_mask (numpy.ndarray): 予測マスク（バイナリ）
    gt_mask (numpy.ndarray): 正解マスク（バイナリ）
    output_path (str): 保存先のパス
    title (str): 画像タイトル
    """
    try:
        # 元画像の読み込み
        original_img = Image.open(original_image_path)
        original_array = np.array(original_img)
        
        # 予測マスクと正解マスクを255にスケーリング（表示用）
        pred_display = pred_mask * 255
        gt_display = gt_mask * 255
        
        # 元画像がグレースケールの場合はRGBに変換
        if len(original_array.shape) == 2:
            original_rgb = np.stack([original_array, original_array, original_array], axis=2)
        else:
            original_rgb = original_array.copy()
        
        # オーバーレイ画像の作成（元画像 + 予測マスク）
        overlay_img = original_rgb.copy()
        
        # 赤色で予測マスクを表示
        if len(overlay_img.shape) == 3 and overlay_img.shape[2] >= 3:
            overlay_img[pred_mask > 0, 0] = 255  # Red channel
            overlay_img[pred_mask > 0, 1] = 0    # Green channel
            overlay_img[pred_mask > 0, 2] = 0    # Blue channel
        
        # 2x2のグリッドで可視化
        fig, axes = plt.subplots(2, 2, figsize=(12, 10))
        
        # 正解マスク
        axes[0, 0].imshow(gt_display, cmap='gray')
        axes[0, 0].set_title('Ground Truth')
        axes[0, 0].axis('off')
        
        # 予測マスク
        axes[0, 1].imshow(pred_display, cmap='gray')
        axes[0, 1].set_title('Predicted Mask')
        axes[0, 1].axis('off')
        
        # 元画像
        axes[1, 0].imshow(original_array, cmap='gray' if len(original_array.shape) == 2 else None)
        axes[1, 0].set_title('Original Image')
        axes[1, 0].axis('off')
        
        # オーバーレイ（元画像 + 予測マスク）
        axes[1, 1].imshow(overlay_img)
        axes[1, 1].set_title('Original with Predicted Mask')
        axes[1, 1].axis('off')
        
        # タイトルがあれば設定
        if title:
            fig.suptitle(title, fontsize=16)
            
        # レイアウト調整と保存
        plt.tight_layout()
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        plt.close(fig)  # メモリ解放のためにfigureを閉じる
        
        print(f"可視化画像を保存しました: {output_path}")
        
    except Exception as e:
        print(f"可視化処理エラー: {e}")
        traceback.print_exc()

# 推論実行用の関数
def run_inference():
    """
    nnUNetを使用して推論を実行する関数
    """
    predictor = nnUNetPredictor(
        tile_step_size=0.5,
        use_gaussian=True,
        use_mirroring=True,
        perform_everything_on_device=False,
        device=torch.device('mps', 0),
        verbose=False,
        verbose_preprocessing=False,
        allow_tqdm=True
    )
    
    # モデルの初期化
    predictor.initialize_from_trained_model_folder(
        model_folder,
        use_folds=(0,),
        checkpoint_name='checkpoint_best.pth',
    )
    
    # 推論を実行
    predictor.predict_from_files(
        input_folder,
        output_folder,
        save_probabilities=False, 
        overwrite=True,
        num_processes_preprocessing=1, 
        num_processes_segmentation_export=1,
        folder_with_segs_from_prev_stage=None, 
        num_parts=1, 
        part_id=0
    )
    print("推論が完了しました。")

# 評価と可視化実行用の関数
def evaluate_and_visualize_results():
    """
    推論結果と正解ラベルを比較して評価指標を計算し、可視化する関数
    """
    # 出力フォルダにあるファイルのリストを取得
    prediction_files = [f for f in os.listdir(output_folder) if f.endswith('.png')]
    
    # 評価結果を保存するリスト
    results = []
    
    for pred_file in prediction_files:
        # 推論結果のファイル名（例: 6870292_16_R_FLOOR-2.png）
        print(f"処理中のファイル: {pred_file}")
        
        # 元画像のパス（例: .../imagesTs/6870292_16_R_FLOOR-2_0000.png）
        # 推論結果は "_0000" が削除されているので、元のファイル名に戻す
        orig_filename = pred_file.rsplit('.', 1)[0] + "_0000.png"
        original_image_path = os.path.join(input_folder, orig_filename)
        
        if not os.path.exists(original_image_path):
            print(f"元画像が見つかりません: {original_image_path}")
            continue
        
        # 正解マスクのパス（例: .../labelsTs/6870292_16_R_FLOOR-2.png）
        # 推論結果と同じファイル名（_0000なし）
        gt_mask_path = os.path.join(gt_mask_folder, pred_file)
        
        if not os.path.exists(gt_mask_path):
            print(f"正解マスクが見つかりません: {gt_mask_path}")
            continue
            
        print(f"元画像: {original_image_path}")
        print(f"予測マスク: {os.path.join(output_folder, pred_file)}")
        print(f"正解マスク: {gt_mask_path}")
        
        # 予測マスクの読み込み
        pred_path = os.path.join(output_folder, pred_file)
        pred_mask = np.array(Image.open(pred_path))
        
        # 正解マスクの読み込み
        gt_mask = np.array(Image.open(gt_mask_path))
        
        # サイズが異なる場合はリサイズ
        if pred_mask.shape != gt_mask.shape:
            print(f"マスクサイズが異なります。予測: {pred_mask.shape}, 正解: {gt_mask.shape}")
            
            # リサイズ処理
            gt_mask_pil = Image.fromarray(gt_mask)
            gt_mask_pil = gt_mask_pil.resize((pred_mask.shape[1], pred_mask.shape[0]), Image.NEAREST)
            gt_mask = np.array(gt_mask_pil)
        
        # メトリクスの計算
        dice, iou, accuracy, precision, recall, pred_binary, gt_binary = calculate_metrics(pred_mask, gt_mask)
        
        # 患者IDと画像番号の抽出
        match = re.search(r'(\d+)_(\d+)_', pred_file)
        if match:
            patient_id = match.group(1)
            img_num = match.group(2)
        else:
            patient_id = "unknown"
            img_num = "unknown"
        
        # 結果を追加
        results.append({
            'Patient_ID': patient_id,
            'img_num': img_num,
            'filename': pred_file,
            'IoU': iou,
            'Dice': dice,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
        })
        
        print(f"処理完了: {pred_file}, Dice: {dice:.4f}, IoU: {iou:.4f}, Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")
        
        # 可視化画像の保存パス
        vis_filename = f"{pred_file.rsplit('.', 1)[0]}_visualization.png"
        vis_path = os.path.join(visualization_folder, vis_filename)
        
        # 可視化を実行
        title = f"Patient: {patient_id}, Image: {img_num}, Dice: {dice:.4f}, IoU: {iou:.4f}"
        create_visualization(original_image_path, pred_binary, gt_binary, vis_path, title)
    
    # 結果をCSVに出力
    if results:
        df = pd.DataFrame(results)
        df.to_csv(csv_output_path, index=False)
        print(f"評価結果をCSVに保存しました: {csv_output_path}")
        
        # 平均値の計算と表示
        print("\n評価指標の平均値:")
        print(f"平均 Dice: {df['Dice'].mean():.4f}")
        print(f"平均 IoU: {df['IoU'].mean():.4f}")
        print(f"平均 Accuracy: {df['Accuracy'].mean():.4f}")
        print(f"平均 Precision: {df['Precision'].mean():.4f}")
        print(f"平均 Recall: {df['Recall'].mean():.4f}")
    else:
        print("評価可能な結果がありませんでした。")

# メイン処理
if __name__ == "__main__":
    # 1. 推論の実行
    print("推論を開始します...")
    run_inference()
    
    # 2. 評価と可視化の実行
    print("評価と可視化を開始します...")
    evaluate_and_visualize_results()
    
    print("すべての処理が完了しました。")

推論を開始します...
perform_everything_on_device=True is only supported for cuda devices! Setting this to False
There are 19 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 19 cases that I would like to predict

Predicting 6870292_16_R_FLOOR-2:
perform_everything_on_device: False


100%|██████████| 1/1 [00:00<00:00,  2.78it/s]


sending off prediction to background worker for resampling and export
done with 6870292_16_R_FLOOR-2

Predicting 6870292_19_R_FLOOR-3:
perform_everything_on_device: False


100%|██████████| 1/1 [00:00<00:00,  6.86it/s]


sending off prediction to background worker for resampling and export
done with 6870292_19_R_FLOOR-3

Predicting 6870292_20_R_FLOOR-3:
perform_everything_on_device: False


...

sending off prediction to background worker for resampling and export
done with 6955010_22_R_FLOOR-3

Predicting 6955010_23_R_FLOOR-3:
perform_everything_on_device: False


100%|██████████| 2/2 [00:00<00:00,  6.69it/s]


sending off prediction to background worker for resampling and export
done with 6955010_23_R_FLOOR-3

Predicting 6955010_24_R_FLOOR-3:
perform_everything_on_device: False


100%|██████████| 2/2 [00:00<00:00,  7.08it/s]


sending off prediction to background worker for resampling and export
done with 6955010_24_R_FLOOR-3
推論が完了しました。
評価と可視化を開始します...
処理中のファイル: 6870292_20_R_FLOOR-3.png
元画像: /Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset005_Orbital_ROI/imagesTs/6870292_20_R_FLOOR-3_0000.png
予測マスク: /Users/yuma/Yuma-Kanematsu/nnUNet/src/results/250422_inference_ROI/6870292_20_R_FLOOR-3.png
正解マスク: /Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset005_Orbital_ROI/labelsTs/6870292_20_R_FLOOR-3.png
予測マスクの値範囲: 0 〜 1
正解マスクの値範囲: 0 〜 1
Intersection: 7885, Union: 10386
予測マスクの陽性ピクセル数: 9697
正解マスクの陽性ピクセル数: 8574
処理完了: 6870292_20_R_FLOOR-3.png, Dice: 0.8631, IoU: 0.7592, Accuracy: 0.9905, Precision: 0.8131, Recall: 0.9196
可視化画像を保存しました: /Users/yuma/Yuma-Kanematsu/nnUNet/src/results/250422_inference_ROI/visualizations/6870292_20_R_FLOOR-3_visualization.png
処理中のファイル: 6870292_19_R_FLOOR-3.png
元画像: /Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset005_Orbital_ROI/imagesTs/6870292_19_R_FLOOR-3_0000.png
予測マスク: /Use

In [30]:

input_jpg_folder = '/Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset002_ForTest_segmenetation/imagesTs_noLabel'
output_png_folder = '/Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset002_ForTest_segmenetation/imagesTs_noLabel_png'
os.makedirs(output_png_folder, exist_ok=True)

# JPGからPNGに変換する関数
def convert_jpg_to_png(input_folder, output_folder, target_size=(512, 512), to_grayscale=True):
    """
    JPG画像をPNG形式に変換し、オプションでグレースケールに変換・リサイズする関数
    
    Parameters:
    input_folder (str): 入力フォルダのパス
    output_folder (str): 出力フォルダのパス
    target_size (tuple): リサイズ後のサイズ (width, height)
    to_grayscale (bool): グレースケールに変換するかどうか
    """
    # 入力フォルダ内のJPGファイルを取得
    jpg_files = [f for f in os.listdir(input_folder) if f.endswith('.jpg')]
    
    for jpg_file in jpg_files:
        # JPGファイルのフルパスを作成
        jpg_path = os.path.join(input_folder, jpg_file)
        
        # 画像を開く
        img = Image.open(jpg_path)
        
        # グレースケールに変換（オプション）
        if to_grayscale:
            img = img.convert('L')
        
        # 画像をリサイズ
        img_resized = img.resize(target_size, Image.Resampling.LANCZOS)
        
        # PNG形式で保存
        png_file = jpg_file.replace('.jpg', '.png')
        png_path = os.path.join(output_folder, png_file)
        img_resized.save(png_path, 'PNG')
        
        print(f"変換・リサイズ完了: {jpg_file} -> {png_file} (サイズ: {target_size}, グレースケール: {to_grayscale})")

# PNGの末尾に"_0000"を追加する関数
def rename_png_files(folder):
    """
    PNGファイルの末尾に"_0000"を追加する関数
    
    Parameters:
    folder (str): フォルダのパス
    """
    png_files = [f for f in os.listdir(folder) if f.endswith('.png')]
    
    for png_file in png_files:
        new_name = png_file.replace('.png', '_0000.png')
        os.rename(os.path.join(folder, png_file), os.path.join(folder, new_name))
        print(f"リネーム完了: {png_file} -> {new_name}")

# JPGからPNGに変換
convert_jpg_to_png(input_jpg_folder, output_png_folder)
# PNGの末尾に"_0000"を追加
rename_png_files(output_png_folder)


変換・リサイズ完了: 513982_17_L_FLOOR-1.jpg -> 513982_17_L_FLOOR-1.png (サイズ: (512, 512), グレースケール: True)
変換・リサイズ完了: 4515533_11_L_FLOOR-2.jpg -> 4515533_11_L_FLOOR-2.png (サイズ: (512, 512), グレースケール: True)
変換・リサイズ完了: 4515533_12_L_FLOOR-2.jpg -> 4515533_12_L_FLOOR-2.png (サイズ: (512, 512), グレースケール: True)
変換・リサイズ完了: 513982_14_L_FLOOR-1.jpg -> 513982_14_L_FLOOR-1.png (サイズ: (512, 512), グレースケール: True)
変換・リサイズ完了: 513982_23_L_FLOOR-1.jpg -> 513982_23_L_FLOOR-1.png (サイズ: (512, 512), グレースケール: True)
変換・リサイズ完了: 513982_27_L_FLOOR-1.jpg -> 513982_27_L_FLOOR-1.png (サイズ: (512, 512), グレースケール: True)
変換・リサイズ完了: 513982_20_L_FLOOR-1.jpg -> 513982_20_L_FLOOR-1.png (サイズ: (512, 512), グレースケール: True)
変換・リサイズ完了: 513982_19_L_FLOOR-1.jpg -> 513982_19_L_FLOOR-1.png (サイズ: (512, 512), グレースケール: True)
変換・リサイズ完了: 513982_24_L_FLOOR-1.jpg -> 513982_24_L_FLOOR-1.png (サイズ: (512, 512), グレースケール: True)
変換・リサイズ完了: 513982_25_L_FLOOR-1.jpg -> 513982_25_L_FLOOR-1.png (サイズ: (512, 512), グレースケール: True)
変換・リサイズ完了: 513982_18_L_FLOOR-1.jpg -> 513982_1

In [47]:
# 環境変数を設定
import os
import re
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image, ImageFile

# 画像に関する問題を回避するための設定
ImageFile.LOAD_TRUNCATED_IMAGES = True

# パスの設定
model_folder = '/Users/yuma/Yuma-Kanematsu/nnUNet/src/results/Dataset004_Orbital_same_testdata/nnUNetTrainer__nnUNetPlans__2d'
input_folder = '/Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset002_ForTest_segmenetation/imagesTs_noLabel_png'

output_folder = '/Users/yuma/Yuma-Kanematsu/nnUNet/src/results/250419_inference_results_NO'
visualization_folder = os.path.join(output_folder, 'visualizations') # 可視化画像の保存フォルダ

# 出力フォルダと可視化フォルダが存在しない場合は作成
os.makedirs(output_folder, exist_ok=True)
os.makedirs(visualization_folder, exist_ok=True)

# ファイル名から患者IDと画像番号を抽出する関数
def extract_patient_info(filename):
    """
    ファイル名から患者IDと画像番号を抽出する関数
    
    Parameters:
    filename (str): ファイル名
    
    Returns:
    tuple: (patient_id, img_num, side, floor)
    """
    # 正規表現でパターンを抽出
    pattern = r'(\d+)_(\d+)_([RL])_FLOOR-(\d+)'
    match = re.search(pattern, filename)
    if match:
        patient_id = match.group(1)
        img_num = match.group(2)
        side = match.group(3)
        floor = match.group(4)
        return patient_id, img_num, side, floor
    else:
        return None, None, None, None

# 可視化関数（正解ラベルなしバージョン）
def create_visualization_no_gt(original_image_path, pred_mask, output_path, title=""):
    """
    元画像、予測マスク、オーバーレイの可視化画像を作成して保存する関数（正解ラベルなし）
    
    Parameters:
    original_image_path (str): 元画像のパス
    pred_mask (numpy.ndarray): 予測マスク（バイナリ）
    output_path (str): 保存先のパス
    title (str): 画像タイトル
    """
    try:
        # 元画像の読み込み
        original_img = Image.open(original_image_path)
        original_array = np.array(original_img)
        
        # 予測マスクを255にスケーリング（表示用）
        pred_display = pred_mask * 255
        
        # 元画像がグレースケールの場合はRGBに変換
        if len(original_array.shape) == 2:
            original_rgb = np.stack([original_array, original_array, original_array], axis=2)
        elif len(original_array.shape) == 3 and original_array.shape[2] == 4:  # RGBA
            original_rgb = original_array[:, :, :3]  # アルファチャンネルを除去
        else:
            original_rgb = original_array.copy()
        
        # オーバーレイ画像の作成（元画像 + 予測マスク）
        overlay_img = original_rgb.copy()
        
        # 赤色で予測マスクを表示
        if len(overlay_img.shape) == 3 and overlay_img.shape[2] >= 3:
            overlay_img[pred_mask > 0, 0] = 255  # Red channel
            overlay_img[pred_mask > 0, 1] = 0    # Green channel
            overlay_img[pred_mask > 0, 2] = 0    # Blue channel
        
        # 1x3のグリッドで可視化（正解ラベルなし）
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        # 元画像
        axes[0].imshow(original_array, cmap='gray' if len(original_array.shape) == 2 else None)
        axes[0].set_title('Original Image')
        axes[0].axis('off')
        
        # 予測マスク
        axes[1].imshow(pred_display, cmap='gray')
        axes[1].set_title('Predicted Mask')
        axes[1].axis('off')
        
        # オーバーレイ（元画像 + 予測マスク）
        axes[2].imshow(overlay_img)
        axes[2].set_title('Original with Predicted Mask')
        axes[2].axis('off')
        
        # タイトルがあれば設定
        if title:
            fig.suptitle(title, fontsize=16)
            
        # レイアウト調整と保存
        plt.tight_layout()
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        plt.close(fig)  # メモリ解放のためにfigureを閉じる
        
        print(f"可視化画像を保存しました: {output_path}")
        
    except Exception as e:
        print(f"可視化処理エラー: {e}")
        import traceback
        traceback.print_exc()

# 1. 推論の実行
def run_inference():
    """
    nnUNetを使用して推論を実行する関数
    """
    predictor = nnUNetPredictor(
        tile_step_size=0.5,
        use_gaussian=True,
        use_mirroring=True,
        perform_everything_on_device=False,
        device=torch.device('mps', 0),
        verbose=False,
        verbose_preprocessing=False,
        allow_tqdm=True
    )
    
    # モデルの初期化
    predictor.initialize_from_trained_model_folder(
        model_folder,
        use_folds=(0,),
        checkpoint_name='checkpoint_best.pth',
    )
    
    # 推論を実行
    predictor.predict_from_files(
        input_folder,
        output_folder,
        save_probabilities=False, 
        overwrite=True,
        num_processes_preprocessing=1, 
        num_processes_segmentation_export=1,
        folder_with_segs_from_prev_stage=None, 
        num_parts=1, 
        part_id=0
    )
    print("推論が完了しました。")

# 2. 可視化の実行（正解ラベルなし）
def visualize_results():
    """
    推論結果を可視化する関数（正解ラベルなし）
    """
    # 出力フォルダにあるセグメンテーション結果ファイルのリストを取得
    prediction_files = [f for f in os.listdir(output_folder) if f.endswith('.png') or f.endswith('.nii.gz')]
    
    for pred_file in prediction_files:
        # ファイル名から拡張子を除去
        file_base = os.path.splitext(pred_file)[0]
        if file_base.endswith('.nii'):  # .nii.gzの場合
            file_base = os.path.splitext(file_base)[0]
        
        # 患者情報を抽出
        patient_id, img_num, side, floor = extract_patient_info(file_base)
        
        if patient_id is None:
            print(f"ファイル名からの情報抽出に失敗: {pred_file}")
            continue
        
        # 元画像のパスを作成
        original_image_filename = f"{patient_id}_{img_num}_{side}_FLOOR-{floor}.png"
        original_image_path = os.path.join(input_folder, original_image_filename)
        
        if not os.path.exists(original_image_path):
            print(f"元画像が見つかりません: {original_image_path}")
            # 一致する画像を検索
            matching_files = [f for f in os.listdir(input_folder) if f.startswith(f"{patient_id}_{img_num}_{side}")]
            if matching_files:
                original_image_path = os.path.join(input_folder, matching_files[0])
                print(f"代替の元画像が見つかりました: {matching_files[0]}")
            else:
                continue
        
        # 予測マスクの読み込み
        pred_path = os.path.join(output_folder, pred_file)
        
        # ファイル拡張子に応じた処理
        if pred_file.endswith('.png'):
            pred_mask = np.array(Image.open(pred_path))
        else:
            # .nii.gzの場合はここで適切な処理を行う
            # 例：SimpleITKなどを使用
            print(f"非PNG形式のファイルはスキップします: {pred_file}")
            continue
        
        # 2値画像に変換
        pred_binary = (pred_mask > 0).astype(np.uint8)
        
        # 可視化画像の保存パス
        vis_filename = f"{patient_id}_{img_num}_{side}_FLOOR-{floor}_visualization.png"
        vis_path = os.path.join(visualization_folder, vis_filename)
        
        # 可視化を実行（正解ラベルなし）
        title = f"Patient: {patient_id}, Image: {img_num}, Side: {side}, Floor: {floor}"
        create_visualization_no_gt(original_image_path, pred_binary, vis_path, title)
    
    print(f"すべての可視化が完了しました。結果は {visualization_folder} に保存されています。")

# メイン処理
if __name__ == "__main__":
    # 1. 推論の実行
    print("推論を開始します...")
    run_inference()
    
    # 2. 可視化の実行（正解ラベルなし）
    print("可視化を開始します...")
    visualize_results()
    
    print("すべての処理が完了しました。")

推論を開始します...
perform_everything_on_device=True is only supported for cuda devices! Setting this to False
There are 19 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 19 cases that I would like to predict

Predicting 4515533_10_L_FLOOR-2:
perform_everything_on_device: False


100%|██████████| 2/2 [00:00<00:00,  7.50it/s]


sending off prediction to background worker for resampling and export
done with 4515533_10_L_FLOOR-2

Predicting 4515533_11_L_FLOOR-2:
perform_everything_on_device: False


100%|██████████| 2/2 [00:00<00:00,  8.98it/s]


sending off prediction to background worker for resampling and export
done with 4515533_11_L_FLOOR-2

Predicting 4515533_12_L_FLOOR-2:
perform_everything_on_device: False


...

sending off prediction to background worker for resampling and export
done with 513982_25_L_FLOOR-1

Predicting 513982_26_L_FLOOR-1:
perform_everything_on_device: False


100%|██████████| 1/1 [00:00<00:00,  8.68it/s]


sending off prediction to background worker for resampling and export
done with 513982_26_L_FLOOR-1

Predicting 513982_27_L_FLOOR-1:
perform_everything_on_device: False


100%|██████████| 1/1 [00:00<00:00,  8.92it/s]


sending off prediction to background worker for resampling and export
done with 513982_27_L_FLOOR-1
推論が完了しました。
可視化を開始します...
元画像が見つかりません: /Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset002_ForTest_segmenetation/imagesTs_noLabel_png/513982_17_L_FLOOR-1.png
代替の元画像が見つかりました: 513982_17_L_FLOOR-1_0000.png
可視化画像を保存しました: /Users/yuma/Yuma-Kanematsu/nnUNet/src/results/250419_inference_results_NO/visualizations/513982_17_L_FLOOR-1_visualization.png
元画像が見つかりません: /Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset002_ForTest_segmenetation/imagesTs_noLabel_png/4515533_11_L_FLOOR-2.png
代替の元画像が見つかりました: 4515533_11_L_FLOOR-2_0000.png
可視化画像を保存しました: /Users/yuma/Yuma-Kanematsu/nnUNet/src/results/250419_inference_results_NO/visualizations/4515533_11_L_FLOOR-2_visualization.png
元画像が見つかりません: /Users/yuma/Yuma-Kanematsu/nnUNet/src/raw_data/Dataset002_ForTest_segmenetation/imagesTs_noLabel_png/4515533_12_L_FLOOR-2.png
代替の元画像が見つかりました: 4515533_12_L_FLOOR-2_0000.png
可視化画像を保存しました: /Users/yuma/Yuma-Kanematsu